In [1]:
import numpy as np
import pandas as pd
import json 

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

bokeh.io.output_notebook()

Loading BokehJS ...

In [79]:
df = pd.read_csv('../Lung_Paper_Images/Quantification/Spleen_Quantification.csv', comment='#')
df.head()

,Date,Tissue,Size Threshold,Gaussian Size,Truncation,Intensity Threshold,Minimum Size,Virus,Animal,Replicate,Count,Cells Quantified,Brightness List,Applied Threshold,Image Multiplication Factor,Minimum Pixel Value,Maximum Pixel Value,Area Threshold,Total Area
0,7/22/2020,Spleen,0.6,5,2,2000,10,AAV5,1,1,43,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.03745250438419489, '2': 0.054506851483...",0.259384,0.507413,-0.126125,0.448653,0.024962,56230025.05
1,7/22/2020,Spleen,0.6,5,2,2000,10,AAV5,1,2,49,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.012048236903504142, '2': 0.04320175573...",0.280709,0.573854,-0.155450,0.486865,0.024342,57470107.19
2,7/22/2020,Spleen,0.6,5,2,2000,10,AAV5,2,1,48,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.08699167774958314, '2': 0.548752784700...",0.276516,0.570794,-0.151739,0.476168,0.022681,48229351.79
3,7/22/2020,Spleen,0.6,5,2,2000,10,AAV5,2,2,42,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.04102931444143054, '2': 0.063642161738...",0.371687,0.574937,-0.226089,0.444059,0.024664,49393907.40
4,7/22/2020,Spleen,0.6,5,2,2000,10,AAV5,3,1,41,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.13521061924446057, '2': 0.169636021930...",0.349644,0.573349,-0.255677,0.542472,0.026923,44840704.89


In [80]:
# Using Determine the transduction as a function of tissue area
df['Cells Transduced per mm2'] = df['Count'] / df['Total Area'] * 1000000

# Get the median brightness from the brightness dictionary
for inds in np.arange(0, len(df)):     
    # Load the brightness dictionary, replacing the single quotation marks in the keys with double quotation marks for json compliance. 
    brightness_dict = json.loads(df.loc[inds, 'Brightness List'].replace("\'", "\""))
    
    # Determine the median of each brightness dictionary and store it in the dataframe
    df.loc[inds, 'Brightness Median'] = np.median(list(brightness_dict.values()))

df['Brightness Median'] = df['Brightness Median'].fillna(0)
    
#Sort the rows 
df = df.sort_values(by=['Virus', 'Animal', 'Replicate'])

# Rename the specific rows for plotting purposes. 
df['Virus'] = df['Virus'].replace('CAPA4','AAV.CAP-A4')

In [81]:
#Group the data by the specific virus and animals
grouped = df.groupby(['Virus','Animal'])

# Make two new dataframes that track cell transduction and median cell brightness respectively
df_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_brightness = grouped['Brightness Median'].mean().reset_index()

In [84]:
#Initialize an output file to save an html of the generated plot
#output_file("../Lung_Paper_Images/Quantification/Lung/lung_transduction_per_sq_mm.html")

# Store the data for plotting
source = ColumnDataSource(df_counts)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Cells Transduced per mm\u00b2')

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.6, range=p.x_range), y='Cells Transduced per mm2', source=source, alpha=1)

# Simply modify plot characteristics 
p.plot_height = 350
p.plot_width = 350

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

p.background_fill_color = None 
p.border_fill_color = None

p.output_backend = "svg"

show(p)

In [85]:
#Initialize an output file to save an html of the generated plot
#output_file("../Lung_Paper_Images/Quantification/Lung/lung_median_brightness.html")

# Store the data for plotting
source = ColumnDataSource(df_brightness)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Median Cell Brightness')

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.6, range=p.x_range), y='Brightness Median', source=source, alpha=1)

p.plot_height = 350
p.plot_width = 350

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

p.background_fill_color = None 
p.border_fill_color = None

p.output_backend = "svg"

show(p)